In [1]:
import json
import numpy as np
import torch
from utils import get_pkl


In [2]:
train_dictionary = get_pkl(f"./data/pkls/train_queries.pkl")
train_queries = get_pkl(f"./data/pkls/train_queries.pkl")

query_names, query_ids = [row[0] for row in train_queries], [row[1] for row in train_queries]
dict_names, dict_ids = [row[0] for row in train_dictionary], [row[1] for row in train_dictionary]


In [3]:

from biosyn.biosyn import BioSyn
biosyn = BioSyn(25, False, 20, 'dmis-lab/biobert-base-cased-v1.1' )

   

c:\Users\antoi\Desktop\thesis\1_2_biosyn_replicate\git_repo_final\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
encoder = biosyn.encoder
tokenizer = biosyn.tokenizer

In [5]:
query_names = query_names[:20]
dict_names = dict_names[:100]
max_length = 25


all_query_names_tokens = tokenizer(query_names, max_length=max_length,padding='max_length', truncation=True, return_tensors='pt' )
all_dict_names_tokens= tokenizer(dict_names, max_length=max_length,padding='max_length', truncation=True, return_tensors='pt')

In [6]:
all_query_names_tokens["input_ids"].shape

torch.Size([20, 25])

In [7]:
topk = 3

cand_tokens_inp = np.empty(shape=( len(all_query_names_tokens["input_ids"]), topk, max_length ), dtype=np.int32)
cand_tokens_att = np.empty(shape=( len(all_query_names_tokens["input_ids"]), topk, max_length ), dtype=np.int32)


for idx in range(len(all_query_names_tokens["input_ids"])):
    cand_idxs = np.random.choice(len(all_dict_names_tokens), size=topk, replace=False)
    cand_tokens_inp[idx] = all_dict_names_tokens["input_ids"][cand_idxs]
    cand_tokens_att[idx] = all_dict_names_tokens["attention_mask"][cand_idxs]



In [8]:
query_embed = encoder(
        input_ids=all_query_names_tokens["input_ids"].squeeze(1),
        attention_mask=all_query_names_tokens["attention_mask"].squeeze(1),
        return_dict=False)[0] # (B, L, H)
query_embed = query_embed[:,0, :].unsqueeze(1)
query_embed.shape

torch.Size([20, 1, 768])

In [9]:
cand_tokens_inp.shape

(20, 3, 25)

In [10]:
cand_tokens_inp1 = torch.tensor(cand_tokens_inp).reshape(-1, max_length)
cand_tokens_att1 = torch.tensor(cand_tokens_att).reshape(-1, max_length)




candidate_embeds = encoder(
    input_ids=cand_tokens_inp1,
    attention_mask=cand_tokens_att1,
    return_dict=False
)[0] #shape B L H

candidate_embeds = candidate_embeds[:,0,:].reshape(20, topk, -1).contiguous()

candidate_embeds.shape

torch.Size([20, 3, 768])

In [11]:
sc = torch.bmm(query_embed, candidate_embeds.permute(0,2,1)).squeeze(1)
sc.shape

torch.Size([20, 3])

In [ ]:
cand_tokens_inp2 = torch.tensor(cand_tokens_inp) # B , top_K, max_length 
cand_tokens_att2 = torch.tensor(cand_tokens_att)


chunk_size = 6
batch_size=20

scores= torch.empty((batch_size, topk))
cands_len = cand_tokens_inp2.shape[0]
offset =0
for start in range(0, cands_len, chunk_size ):
    end = min(start + chunk_size, cands_len)
    chunk_inp = cand_tokens_inp2[start:end, :, :].reshape(-1, max_length) # chunk_size * topk, L
    chunk_att = cand_tokens_att2[start:end, : , :].reshape(-1, max_length)
    chunk_cand_emb = encoder(input_ids=chunk_inp, attention_mask=chunk_att, return_dict=False)[0][:, 0, :] #cls #chunk_size * topk,  H
    chunk_cand_emb = chunk_cand_emb.reshape((end-start), topk, -1).transpose(1,2)
    chunk_query_embed = query_embed.squeeze(1)[start:end, :]
    # print(f"b1: {chunk_query_embed.unsqueeze(1).shape} , b2: {chunk_cand_emb.shape}")
    score= torch.bmm(chunk_query_embed.unsqueeze(1), chunk_cand_emb).squeeze(1)
    scores[offset : offset + (end-start)] = torch.tensor(score.detach().numpy())
    
  
    offset = end
    del score

print(scores.shape)

In [ ]:
torch.allclose(scores, sc)

In [ ]:
scores

In [ ]:
sc